### [ RAG 구현 절차 ]
```
1. 문서의 내용을 읽는다(document_loader를 이용)
    (1)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
    (2)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
    %pip install --upgrade --quiet  docx2txt

2. 문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)
    (1)	 https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
    %pip install -qU langchain-text-splitters

3. 쪼갠 문서를 임베딩하여 vector database에 넣음
    (1)	OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
    (2)	https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
    %pip install -q langchain-chroma

4. 질문을 이용해 유사도 검색

5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)
    (1)	https://python.langchain.com/v0.2/docs/tutorials/rag/
    %pip install -q langchain langchainhub

https://smith.langchain.com/에서 key생성 .env key 추가 (LANGCHAIN_API_KEY)
```

# 2. 문서를 쪼개면서 읽기(O)

In [ ]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter( # 문서를 쪼개는 기준이 문자수
                                                chunk_size=1500, # 문자를 1500글자씩 쪼갬
                                                chunk_overlap=200,
                                                )
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 runtime : ', runtime)

In [ ]:
# 나눠진 chunk 갯수
len(document)

In [ ]:
len(document[0].page_content)

In [ ]:
# chunk의 글자수
# [len(doc.page_content) for doc in document]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

# 3. 나눈 문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : upstage의 text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [1]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage/#using-with-vector-store

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [ ]:
doc_result = embeddings.embed_documents(
    ["소득세법",documents[0].page_content]
)
print(len(doc_result), len(doc_result[0]), len(doc_result[1]))

In [2]:
%%time
from langchain_chroma import Chroma
# 데이터를 처름 저장할 때
# database = Chroma.from_documents(
#     documents=documents,
#     embedding=embeddings,
#     collection_name="tax_collection", # 생략시 이름이 랜덤으로 적용
#     persist_directory="./chroma_upstage",     # 생략시 로컬데이터베이스에 저장안됨. 프로그램 종료시 db날라감
# )
# 이미 저장된 vecter DB를 사용할 때
database = Chroma(
    embedding_function=embeddings,
    collection_name="tax_collection",
    persist_directory="./chroma_upstage",
)

CPU times: total: 562 ms
Wall time: 773 ms


# 4. vecter DB에 질문과 유사도 검색(답변 생성을 위한 retriever)

In [3]:
query = '연봉 5천만원인 직장인의 소득세는 얼마 인가요?'
retrieved_docs = database.similarity_search(query,
                                            k=3, # 기본 k는 4이며, 최대 k는 100이다.
                                            )

In [ ]:
retrieved_docs[0]

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1-nano')

In [5]:
prompt = f'''[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 징문에 답변해 주세요
[context]는 다음과 같습니다
{retrieved_docs}
Question: {query}'''

In [6]:
ai_message = llm.invoke(prompt)

In [7]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하려면, 먼저 근로소득공제액과 기타 공제액을 고려해야 합니다. 일반적으로 근로소득공제와 표준세율을 적용하여 산출하는 방식으로 계산됩니다. 아래는 대략적인 계산 과정입니다.

1. **총급여액(연봉)** : 50,000,000원

2. **근로소득공제** : 제47조에 따라 최대 2천만원까지 공제됩니다.
   - 5천만원의 근로소득공제는 대략적으로 계산하면 약 1,750만원에서 2천만원 정도로 보시면 됩니다.

3. **과세표준** : 총급여액에서 근로소득공제를 차감
   - 50,000,000원 - 약 19,500,000원(근로소득공제) ≈ 30,500,000원

4. **기본 공제와 인적공제 등 기타 공제** : 인적공제(본인공제 150만원, 부양가족 공제 등), 기본공제는 별도 고려하지 않으면, 과세표준이 3,050만원 수준입니다.

5. **세율** : 소득세는 누진세율 적용(2023년 기준)
   - 1,200만원 이하: 6%
   - 1,200만원 초과 ~ 4,600만원 이하: 15%

6. **대략적인 세액 계산** :
   - 1,200만원 x 6% = 72만원
   - (3,050만원 - 1,200만원) x 15% = 271.5만원
   - 합계: 72만원 + 271.5만원 = 약 343.5만원

이 값은 기본적인 예상치이며, 상세 계산 시 부양가족, 보험료, 기타 공제사항 등에 따라 다소 차이가 있을 수 있습니다.

**요약:**  
연봉 5천만원인 직장인의 대략적인 소득세는 약 **340만원 내외**입니다.

추가로 구체적인 공제항목이나 세액감면이 있으면 최종 금액이 달라질 수 있으니, 상세한 계산을 원하시면 공제내역을 알려주시기 바랍니다.


# 5. Augmentation을 위한 제공되는 Prompt 활용하여 langchain으로 답변 생성

In [8]:
query = '연봉 5천만원인 직장인의 소득세는 얼마 인가요?'
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### RetrievalQA를 통해 LLM전달(create_retrieval_chain이 대체)
```
query -> retriever전달(벡터 검색 수행) -> retriever문서 -> promp의 {context}에 삽입
-> query -> prompt의 {question}에 삽입
```

In [9]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs = {"prompt": prompt},
)

In [10]:
ai_message = qa_chain.invoke({'query' : query})

In [11]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마 인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 74만원입니다. 이는 근로소득공제와 근로소득세액공제 등을 고려한 일반적인 계산 결과입니다. 정확한 세액은 상세한 소득공제 항목에 따라 다를 수 있습니다.'}